In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import argparse
from itertools import combinations

In [2]:
# 초성 리스트. 00 ~ 18 총 19개
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20 총 21개
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음) 총 28개
JONGSUNG_LIST = ['J없음', 'Jㄱ', 'Jㄲ', 'Jㄳ', 'Jㄴ', 'Jㄵ', 'Jㄶ', 'Jㄷ', 'Jㄹ', 'Jㄺ', 'Jㄻ', 'Jㄼ', 'Jㄽ', 'Jㄾ', 'Jㄿ', 'Jㅀ', 'Jㅁ', 'Jㅂ', 'Jㅄ', 'Jㅅ', 'Jㅆ', 'Jㅇ', 'Jㅈ', 'Jㅊ', 'Jㅋ', 'Jㅌ', 'Jㅍ', 'Jㅎ']
#숫자 리스트 00 ~ 09 총 10개
NUMBER_LIST = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
#알파벳 00 ~ 25 총 25개
ALPHABET_LIST = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
#특수문자 00 ~ 18 총 19개
SPECIAL_CHAR_LIST = ['!', '\"', '#', '$', '%', '&', '\'', '?', '@', '*', '+', ',', '-', '.', '/', '~', ' ', ':', '^']
#추가 총 4개
SINGLE_CHAR_LIST = ['ㄳ', 'ㄵ', 'ㅄ','ㄺ']
#총 127개

ALL_CHAR = []
ALL_CHAR = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
ALL_CHAR += JUNGSUNG_LIST
ALL_CHAR += JONGSUNG_LIST
ALL_CHAR += NUMBER_LIST
ALL_CHAR += ALPHABET_LIST
ALL_CHAR += SPECIAL_CHAR_LIST
ALL_CHAR += SINGLE_CHAR_LIST

In [3]:
#문자열 데이터가 들어오면 one hot encoding을해서 리스트를 반환.
def onehotencoding(string):
    return_list = []
    tmp_list = []
    check = 0
    for i in range(len(string)):
        for k in range(len(ALL_CHAR)):
            if(ALL_CHAR[k] == string[i]):
                tmp_list.append(1)
                check = 1
            else:
                tmp_list.append(0)
        if(check == 1):
            return_list.append(tmp_list)
            tmp_list = []
            check = 0
        else:
            tmp_list = []
    return return_list

In [4]:
#자모단위로 분해
def tokenize(word):
    r_lst = []
    for w in list(word.strip()):
        ## 영어인 경우 구분해서 작성함. 
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜. 
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append(CHOSUNG_LIST[ch1])
            r_lst.append(JUNGSUNG_LIST[ch2])
            if ch3 != 0:
                r_lst.append(JONGSUNG_LIST[ch3])
        elif 'A'<=w<='Z':
            ch = (ord(w) - ord('A'))
            r_lst.append(ALPHABET_LIST[ch])
        elif 'a'<=w<='z':
            ch = (ord(w) - ord('a'))
            r_lst.append(ALPHABET_LIST[ch])
        else:
            r_lst.append(w)
         
    
    return r_lst

In [5]:
#데이터 크기가 n보다 크면 n까지 자르고, n보다 작으면 n크기까지 zero padding을 함.
def max_resize_data(data):
    n = 200
    if(len(data) > n):
        data = data[:n]
    return data

def zero_padding_resize_data(data):
    zero_list = [0 for k in range(127)]
    for i in range(len(data), 200):
        data.append(zero_list)
    return data

In [6]:
class Text_CNN(nn.Module):
    def __init__(self):
        super(Text_CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 127, out_channels = 512, kernel_size = 6, stride=1),
            #[batch_size, 512, 195]
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=1)
            #[batch_size, 512, 193]
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 9, stride=1),
            #[batch_size, 512, 185]
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=1)
            #[batch_size, 512, 183]
        ) 
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 12, stride=1),
            #[batch_size, 512, 172]
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=1)
            #[batch_size, 512, 170]
        )
        self.fc1 = nn.Sequential(
            nn.Linear(in_features=512*170, out_features=1024),
            #[batch_size, 86016] -> [batch_size, 1024]
            nn.ReLU(),
            #nn.Dropout(p = 0.5) #테스트 타임떄는 dropout 제거
        )
        self.fc2 = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1024),
            #[batch_size, 1024] -> [batch_size, 1024]
            nn.ReLU(),
           #nn.Dropout(p=0.5) #테스트 타임떄는 dropout 제거
        )
        self.fc3 = nn.Linear(in_features=1024, out_features=1)
        #[batch_size, 1024] -> [batch_size, 2]
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        #[batch_size, 512, 120]
        x = self.conv2(x)
        #[batch_size, 512, 110]
        x = self.conv3(x)
        #[batch_size, 512, 97]
        
        # collapse
        x = x.view(x.size(0), -1) #[batch_size, 49664]
        # linear layer
        x = self.fc1(x) #[batch_size, 1024]
        # linear layer
        x = self.fc2(x) #[batch_size, 1024]
        # linear layer
        x = self.fc3(x) #[batch_size, 2]
        # output layer
        x = self.sigmoid(x)
        
        return x

In [7]:
def isbadword(s_list):
    li_str = []
    li_num = []
    for i in range(0, len(s_list)):
        a = list(combinations(s_list, i+1))
        for j in range(0, len(a)):
            str1 = ' '.join(a[j])
            li_str.append(str1)
            li_num.append(make_output(str1))
    
    li_str.reverse()
    li_num.reverse()
    return li_str, li_num

In [8]:
def choose_one_string(li_str, li_num, s_list):
    for i in range(0, len(li_num)):
        #if( make_output(s)-0.5 > li_num):
        if( 0.4 > li_num[i]):
            a = li_str[i].split()
            
            for j in range(0, len(s_list)):
                
                if( a.count(s_list[j]) == 0):
                    len_l = len(s_list[j])
                    s_list.remove(s_list[j])
                    num_s = "*"
                    for k in range(1, len_l):
                        num_s = num_s + "*"
                    s_list.insert(j, num_s)
                    len_l = 0
                    
            return ' '.join(s_list)
    
    tmp = []
    for i in range(0, len(s_list)):
        num_s = "*"
        for j in range(1, len(s_list[i])):
            num_s = num_s + "*"
        tmp.insert(i, num_s)
    return ' '.join(tmp)

In [9]:
def check_isbad(num):
    if(num > 0.6):
        return True
    else:
        return False

In [10]:
def make_output(string):
    x = zero_padding_resize_data(max_resize_data(onehotencoding(tokenize(string))))
    x = np.asarray(x).astype('float32')
    x = torch.from_numpy(x)
    x = x.reshape(1, 200, 127)
    x = x.permute(0, 2, 1)
    x = x.to(device)
    output = load_model.forward(x)
    return output.item()

In [14]:
#모델 불러오기
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
load_model = Text_CNN().to(device)
load_model.load_state_dict(torch.load('./text_cnn_pytorch_V4.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [27]:
s = "안 녕 하 세 요 시발 반 갑 습니 다 개새끼야"

In [28]:
print("욕일 확률 : {}".format(make_output(s)*100))

욕일 확률 : 99.56751465797424


In [29]:
if(check_isbad(make_output(s)) == True):#욕이라고 판단했을때, (기준은 60%이상)
    s_list = []
    s_list = s.split()
    li_str, li_num = isbadword(s_list)
    print_s = choose_one_string(li_str, li_num, s_list)
    for i in range(0, len(li_num)):
        print(li_str[i], "     ", li_num[i])
else: #욕이 아니라고 판단했을때
    print_s = s

안 녕 하 세 요 시발 반 갑 습니 다 개새끼야       0.9956751465797424
녕 하 세 요 시발 반 갑 습니 다 개새끼야       0.9941457509994507
안 하 세 요 시발 반 갑 습니 다 개새끼야       0.9940325617790222
안 녕 세 요 시발 반 갑 습니 다 개새끼야       0.9960197806358337
안 녕 하 요 시발 반 갑 습니 다 개새끼야       0.9965147376060486
안 녕 하 세 시발 반 갑 습니 다 개새끼야       0.9964370727539062
안 녕 하 세 요 반 갑 습니 다 개새끼야       0.9799287915229797
안 녕 하 세 요 시발 갑 습니 다 개새끼야       0.9944881796836853
안 녕 하 세 요 시발 반 습니 다 개새끼야       0.9952648878097534
안 녕 하 세 요 시발 반 갑 다 개새끼야       0.9952757358551025
안 녕 하 세 요 시발 반 갑 습니 개새끼야       0.9964168071746826
안 녕 하 세 요 시발 반 갑 습니 다       0.9259961843490601
하 세 요 시발 반 갑 습니 다 개새끼야       0.9969532489776611
녕 세 요 시발 반 갑 습니 다 개새끼야       0.9962263107299805
녕 하 요 시발 반 갑 습니 다 개새끼야       0.9958404898643494
녕 하 세 시발 반 갑 습니 다 개새끼야       0.9960582256317139
녕 하 세 요 반 갑 습니 다 개새끼야       0.9828419089317322
녕 하 세 요 시발 갑 습니 다 개새끼야       0.9947291016578674
녕 하 세 요 시발 반 습니 다 개새끼야       0.9952753782272339
녕 하 세 요 시발 반 갑 다 개새끼야       0.9949358701705933
녕 하 세 요 시발 반 갑 습니 개새끼

안 녕 하 시발 반 습니 다 개새끼야       0.995913565158844
안 녕 하 시발 반 갑 다 개새끼야       0.9973042011260986
안 녕 하 시발 반 갑 습니 개새끼야       0.9955417513847351
안 녕 하 시발 반 갑 습니 다       0.910464346408844
안 녕 하 요 갑 습니 다 개새끼야       0.9886431097984314
안 녕 하 요 반 습니 다 개새끼야       0.9895127415657043
안 녕 하 요 반 갑 다 개새끼야       0.9855637550354004
안 녕 하 요 반 갑 습니 개새끼야       0.9891921877861023
안 녕 하 요 반 갑 습니 다       0.010655988939106464
안 녕 하 요 시발 습니 다 개새끼야       0.9976963400840759
안 녕 하 요 시발 갑 다 개새끼야       0.9973439574241638
안 녕 하 요 시발 갑 습니 개새끼야       0.995731770992279
안 녕 하 요 시발 갑 습니 다       0.922094464302063
안 녕 하 요 시발 반 다 개새끼야       0.9973568916320801
안 녕 하 요 시발 반 습니 개새끼야       0.9961752891540527
안 녕 하 요 시발 반 습니 다       0.935869038105011
안 녕 하 요 시발 반 갑 개새끼야       0.9971987009048462
안 녕 하 요 시발 반 갑 다       0.9375040531158447
안 녕 하 요 시발 반 갑 습니       0.9216268658638
안 녕 하 세 갑 습니 다 개새끼야       0.9902758598327637
안 녕 하 세 반 습니 다 개새끼야       0.9887162446975708
안 녕 하 세 반 갑 다 개새끼야       0.985149621963501
안 녕 하 세 반 갑 습니 개새끼야       0.

안 세 요 반 갑 습니 개새끼야       0.9801658391952515
안 세 요 반 갑 습니 다       0.005564531777054071
안 세 요 시발 습니 다 개새끼야       0.9977707862854004
안 세 요 시발 갑 다 개새끼야       0.9967923760414124
안 세 요 시발 갑 습니 개새끼야       0.9967706203460693
안 세 요 시발 갑 습니 다       0.938212513923645
안 세 요 시발 반 다 개새끼야       0.9965638518333435
안 세 요 시발 반 습니 개새끼야       0.9969087243080139
안 세 요 시발 반 습니 다       0.9449225068092346
안 세 요 시발 반 갑 개새끼야       0.9951283931732178
안 세 요 시발 반 갑 다       0.9395676255226135
안 세 요 시발 반 갑 습니       0.9303026795387268
안 하 반 갑 습니 다 개새끼야       0.9816038608551025
안 하 시발 갑 습니 다 개새끼야       0.9965225458145142
안 하 시발 반 습니 다 개새끼야       0.9970732927322388
안 하 시발 반 갑 다 개새끼야       0.9954078793525696
안 하 시발 반 갑 습니 개새끼야       0.9972509741783142
안 하 시발 반 갑 습니 다       0.9334779381752014
안 하 요 갑 습니 다 개새끼야       0.9825301766395569
안 하 요 반 습니 다 개새끼야       0.9821908473968506
안 하 요 반 갑 다 개새끼야       0.9856727123260498
안 하 요 반 갑 습니 개새끼야       0.9801295399665833
안 하 요 반 갑 습니 다       0.006322070490568876
안 하 요 시발 습니 다 개새끼야  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.9968095421791077
안 녕 하 시발 습니 다       0.9556538462638855
안 녕 하 시발 갑 개새끼야       0.997075080871582
안 녕 하 시발 갑 다       0.9464268684387207
안 녕 하 시발 갑 습니       0.9205977320671082
안 녕 하 시발 반 개새끼야       0.996731162071228
안 녕 하 시발 반 다       0.9423366189002991
안 녕 하 시발 반 습니       0.9294013977050781
안 녕 하 시발 반 갑       0.9192172884941101
안 녕 하 요 다 개새끼야       0.9958078861236572
안 녕 하 요 습니 개새끼야       0.991005539894104
안 녕 하 요 습니 다       0.020068833604454994
안 녕 하 요 갑 개새끼야       0.9932318925857544
안 녕 하 요 갑 다       0.010956993326544762
안 녕 하 요 갑 습니       0.012059607543051243
안 녕 하 요 반 개새끼야       0.9924151301383972
안 녕 하 요 반 다       0.017078040167689323
안 녕 하 요 반 습니       0.03194102644920349
안 녕 하 요 반 갑       0.022950848564505577
안 녕 하 요 시발 개새끼야       0.998896598815918
안 녕 하 요 시발 다       0.9505290389060974
안 녕 하 요 시발 습니       0.9569534659385681
안 녕 하 요 시발 갑       0.9593672156333923
안 녕 하 요 시발 반       0.9608751535415649
안 녕 하 세 다 개새끼야       0.9969130754470825
안 녕 하 세 습니 개새끼야       0.9957184195518494


녕 세 반 갑 습니       0.00871319230645895
녕 세 시발 다 개새끼야       0.9997108578681946
녕 세 시발 습니 개새끼야       0.9986549615859985
녕 세 시발 습니 다       0.9664177894592285
녕 세 시발 갑 개새끼야       0.9992658495903015
녕 세 시발 갑 다       0.9471310377120972
녕 세 시발 갑 습니       0.9443322420120239
녕 세 시발 반 개새끼야       0.9992939233779907
녕 세 시발 반 다       0.9572283029556274
녕 세 시발 반 습니       0.9596748352050781
녕 세 시발 반 갑       0.9446921348571777
녕 세 요 다 개새끼야       0.9977788329124451
녕 세 요 습니 개새끼야       0.997710108757019
녕 세 요 습니 다       0.014240278862416744
녕 세 요 갑 개새끼야       0.9984246492385864
녕 세 요 갑 다       0.011839752085506916
녕 세 요 갑 습니       0.01299233827739954
녕 세 요 반 개새끼야       0.9977656602859497
녕 세 요 반 다       0.009565207175910473
녕 세 요 반 습니       0.00767632806673646
녕 세 요 반 갑       0.008375952951610088
녕 세 요 시발 개새끼야       0.9997519850730896
녕 세 요 시발 다       0.935916543006897
녕 세 요 시발 습니       0.9681546688079834
녕 세 요 시발 갑       0.960779070854187
녕 세 요 시발 반       0.959969162940979
녕 하 습니 다 개새끼야       0.997417807

안 녕 시발 다 개새끼야       0.999554455280304
안 녕 시발 습니 개새끼야       0.9987836480140686
안 녕 시발 습니 다       0.9760358929634094
안 녕 시발 갑 개새끼야       0.9988846182823181
안 녕 시발 갑 다       0.9623886942863464
안 녕 시발 갑 습니       0.9559677243232727
안 녕 시발 반 개새끼야       0.998779833316803
안 녕 시발 반 다       0.9634305238723755
안 녕 시발 반 습니       0.9634438157081604
안 녕 시발 반 갑       0.9520142674446106
안 녕 요 다 개새끼야       0.9980149269104004
안 녕 요 습니 개새끼야       0.9957700371742249
안 녕 요 습니 다       0.012696764431893826
안 녕 요 갑 개새끼야       0.9981182813644409
안 녕 요 갑 다       0.010896004736423492
안 녕 요 갑 습니       0.013938655145466328
안 녕 요 반 개새끼야       0.99788898229599
안 녕 요 반 다       0.017703000456094742
안 녕 요 반 습니       0.03261037543416023
안 녕 요 반 갑       0.015315734781324863
안 녕 요 시발 개새끼야       0.9995598196983337
안 녕 요 시발 다       0.9375776052474976
안 녕 요 시발 습니       0.95845627784729
안 녕 요 시발 갑       0.9566302299499512
안 녕 요 시발 반       0.9542829394340515
안 녕 세 다 개새끼야       0.9986045956611633
안 녕 세 습니 개새끼야       0.997694909

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




세 요       0.018837442621588707
하 개새끼야       0.9927305579185486
하 다       0.011889932677149773
하 습니       0.01040111668407917
하 갑       0.012298544868826866
하 반       0.011784493923187256
하 시발       0.9561063051223755
하 요       0.011852690950036049
하 세       0.019776910543441772
녕 개새끼야       0.9953899383544922
녕 다       0.01244034618139267
녕 습니       0.012071758508682251
녕 갑       0.018773142248392105
녕 반       0.017108075320720673
녕 시발       0.9571663737297058
녕 요       0.012030724436044693
녕 세       0.02313978783786297
녕 하       0.013475525192916393
안 개새끼야       0.9955874681472778
안 다       0.011868628673255444
안 습니       0.010621048510074615
안 갑       0.011963769793510437
안 반       0.011780430562794209
안 시발       0.9390164613723755
안 요       0.01180122047662735
안 세       0.013895757496356964
안 하       0.011787925846874714
안 녕       0.017540788277983665
개새끼야       0.975642204284668
다       0.011842228472232819
습니       0.017538446933031082
갑       0.011842106468975544
반       0.01185

In [30]:
print_s

'안 녕 하 세 요 ** 반 갑 습니 다 ****'